In [1]:
import pandas as pd
import geopandas as gpd

### Grab HCDN Basins within Colorado, Utah, and Wyoming
- gathered from https://water.usgs.gov/GIS/metadata/usgswrd/XML/gagesII_Sept2011.xml
- Basins also grabbed from https://water.usgs.gov/GIS/metadata/usgswrd/XML/gagesII_Sept2011.xml


In [ ]:
df = pd.read_excel('../../data/gagesII_sept30_2011_conterm.xlsx')
gdf = gpd.GeoDataFrame(df,crs="EPSG:4326", geometry=gpd.points_from_xy(x=df.LNG_GAGE, y=df.LAT_GAGE))
gdf_filtered = gdf[(gdf['STATE'].isin(['CO','UT','WY'])) &
                    ((gdf['HCDN-2009'] == 'yes') |
                     (gdf['OLD_HCDN'] == 'yes'))].set_index('STAID',drop=True)

### Filter to Basins within the UCRB

In [ ]:
ucrb_polygon = gpd.read_file('../multisite/polygons/Upper_Colorado_River_Basin_Boundary.json')
gdf_filtered_ucrb = gdf_filtered.clip(ucrb_polygon).to_crs('EPSG:5070')
gdf_filtered_ucrb.to_file('./basin_characteristics/ucrb_hcdn_stations.json',driver='GeoJSON')

### Grab Snotel Stations in UCRB

In [12]:
# Get SNTL data
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"

# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)

gdf_sntl = gdf_sntl.to_crs('4326')

In [15]:
gdf_sntl_ucrb = gdf_sntl.clip(ucrb_polygon)
# gdf_sntl_ucrb.to_file('ucrb_SNOTEL_locations.json', driver='GeoJSON')